# Scientific Repository by Hayes 

The Scientific Repository program is a three-part system: a backend class for data operations, a user interface with built-in error handling for seamless user interactions, and a suite of unit tests for reliability check. It is designed to effectively manage scientific resources, ensuring both accuracy and user-friendly experience for managing a scientific repository.

# Overview of the ScientificRepository Class

The ScientificRepository class functions as a robust backend engine, designed to manage a variety of scientific resources such as books, datasets, and articles. As a backend system, it provides the structure and methods for storing, manipulating, and retrieving data, but does not directly interact with users. Instead, it would be used in conjunction with a frontend to provide user interfaces. Here are its key features:

Repository Initialization and Persistence (__init__, _load_repository, save_repository): The ScientificRepository class manages a collection of scientific materials stored in a JSON file, which is a standard, human-readable data exchange format. When a ScientificRepository object is initialized with a filename (self.filename), it checks if a file with that name already exists. If it does, the repository is loaded from this file. If it doesn't, an empty repository (self.filename) is initialized. Any time a change is made to the repository (such as adding or removing materials), the updated repository is saved back to the JSON file, effectively creating the file if it didn't exist before. This ensures that all modifications to the repository are retained. By default, the JSON file is located in the same directory as the Python script, unless a different path is specified during initialization.

Material Management (add_material, search_material, search_favorites, sort_materials, remove_material, update_material, batch_add_materials): These methods provide a comprehensive suite of management tools for the scientific materials within the repository. They facilitate the adding of single or multiple materials, offer advanced search capabilities for materials based on specific criteria, and allow for the sorting of materials. Importantly, they also provide for the removal or alteration of a material, and the ability to mark a material as a favorite.

Data Analysis (generate_report): This method generates an insightful report, offering a detailed overview of the materials stored in the repository. The report includes the total count of materials, a breakdown of materials by type and field, a tally of unique authors and keywords, a year-by-year distribution of materials, and a list of the most frequently utilized keywords. This is especially useful for gaining a high-level understanding of the repository's content.

Material Verification (material_exists, material_exists_by_id): These methods allow for verification if a material exists in the repository either by its attributes or a unique id. This is important for preventing the duplication of materials and for allowing specific materials to be easily retrieved.

Favorite Management (mark_as_favorite): This method provides the ability to mark a material as a favorite, which could be used as a part of user personalization features in a frontend application.

Material Access (get_all_repository): This method returns all the materials in the repository, allowing complete access to the data stored. This can be useful for bulk operations or for providing data to other systems.

In [ ]:
import json
import os
import unittest
import uuid
from collections import Counter
from datetime import datetime

class ScientificRepository:
    
    """Initialize the repository with the filename to load and save data."""
    def __init__(self, filename):
        self.filename = filename
        self.repository = []
        self._load_repository()

    def _load_repository(self):
        """Load repository from file or initialize an empty one if file doesn't exist."""
        if os.path.exists(self.filename):
            with open(self.filename, 'r') as file:
                self.repository = json.load(file)
        else:
            self.repository = []

    def save_repository(self):
        """Save repository to file."""
        with open(self.filename, 'w') as file:
            json.dump(self.repository, file)

    def add_material(self, material):
        """Add a new material to the repository and save the updated repository."""
        self.repository.append(material)
        self.save_repository()

    def search_material(self, key, value):
        """Search for materials based on a key-value pair."""
        if key == 'keywords' or key == 'authors':
            return [material for material in self.repository if value in material.get(key, [])]
        else:
            return [material for material in self.repository if material.get(key) == value]

    def search_favorites(self):
        """Search for all materials that are marked as favorites."""
        return [material for material in self.repository if material.get('favorite')]
    
    def sort_materials(self, key, order='asc'):
        """Sort the materials based on a given key. Returns a new sorted list."""
        if key == 'publication_date':
            sorted_list = sorted(self.repository, key=lambda x: datetime.strptime(x.get(key, ''), '%Y-%m-%d'))
        else:
            sorted_list = sorted(self.repository, key=lambda x: x.get(key))

        if order == 'desc':
            sorted_list.reverse()

        return sorted_list


    def remove_material(self, id):
        """Remove a material from the repository based on its id."""
        # Check if the material exists in the repository
        if any(material.get('id') == id for material in self.repository):
            self.repository = [material for material in self.repository if material.get('id') != id]
            self.save_repository()
            print("Material removed successfully!")
        else:
            print("Invalid ID. The material does not exist.")

    def update_material(self, id, key, new_value):
        """Update a specific attribute (key) of a material with a new value."""
        for material in self.repository:
            if material.get('id') == id:
                material[key] = new_value
                self.save_repository()
                return True  # Return True when the material is successfully updated

        return False  # Return False when no material with the given id is found

    def batch_add_materials(self, materials):
        """Add multiple materials to the repository at once."""
        self.repository.extend(materials)
        self.save_repository()

    def mark_as_favorite(self, id):
        """Mark a material as favorite."""
        self.update_material(id, 'favorite', True)
        
    def get_all_repository(self):
        """
        This method returns all the materials currently in the repository.
        It doesn't modify the repository, but allows for inspection of its contents.

        Returns:
            list: A list of all materials in the repository.
        """
        return self.repository

    def generate_report(self):
        # Initialize counters
        type_counter = {}
        field_counter = {}
        author_counter = Counter()
        keyword_counter = Counter()
        year_counter = Counter()
        earliest_date = None
        latest_date = None

        for material in self.repository:
            # Count types
            type = material['type']
            if type in type_counter:
                type_counter[type] += 1
            else:
                type_counter[type] = 1

            # Count fields
            field = material['associated_field']
            if field in field_counter:
                field_counter[field] += 1
            else:
                field_counter[field] = 1

            # Count unique authors and keywords
            for author in material['authors']:
                author_counter[author] += 1
            for keyword in material['keywords']:
                keyword_counter[keyword] += 1

            # Update earliest and latest dates
            publication_date = datetime.strptime(material['publication_date'], '%Y-%m-%d')
            year_counter[publication_date.year] += 1
            if not earliest_date or publication_date < earliest_date:
                earliest_date = publication_date
            if not latest_date or publication_date > latest_date:
                latest_date = publication_date

        # Generate report
        report = "Scientific Repository Summary:\n"
        report += f"Total number of materials: {len(self.repository)}\n"
        report += "Types:\n" + '\n'.join(f"{type}: {count}" for type, count in type_counter.items())
        report += "\nFields:\n" + '\n'.join(f"{field}: {count}" for field, count in field_counter.items())
        report += f"\nNumber of unique authors: {len(author_counter)}"
        report += f"\nNumber of unique keywords: {len(keyword_counter)}"
        report += "\nMaterials by year:\n" + '\n'.join(f"{year}: {count}" for year, count in year_counter.items())
        report += f"\nEarliest publication date: {earliest_date.strftime('%Y-%m-%d')}"
        report += f"\nLatest publication date: {latest_date.strftime('%Y-%m-%d')}"
        report += "\nMost common keywords:\n" + '\n'.join(f"{keyword}: {count}" for keyword, count in keyword_counter.most_common(5))

        return report

    def material_exists(self, material):
        """
        Checks if a given material exists in the repository based on its details.
    
        Returns True if the material exists, False otherwise."""
        for existing_material in self.repository:
            if (existing_material['type'].lower() == material['type'].lower() and
                existing_material['title'].lower() == material['title'].lower() and
                existing_material['authors'] == material['authors'] and
                existing_material['publication_date'] == material['publication_date']):
                return True
        return False
    def material_exists_by_id(self, id):
        """
        Checks if a given material exists in the repository based on its ID.
    
        Returns True if the material exists, False otherwise."""
        for existing_material in self.repository:
            if existing_material['id'] == id:
                return True
        return False

# Overview of the ScientificRepository Class

The ScientificRepositoryUI class is engineered to offer an intuitive user interface for interacting with a collection of scientific materials, such as academic papers, datasets, books, and more. This class fulfills a range of duties:

Initiating the Repository: The class begins by initializing an instance of the ScientificRepository with a given repository file. It also computes the next ID for the addition of new materials.

Visualizing Materials: The class presents a method to display all materials in the repository, providing a user-friendly overview of the contents.

Adding Materials to the Repository: The class includes a function to add a new material to the repository. It prompts the user for the necessary details, then increments the next available ID and adds the new material.

Removing Materials from the Repository: The class features a method to remove an existing material by its ID, offering users the flexibility to manage their repository.

Updating Existing Materials: The class enables users to update the details of a material. It requests the user to furnish the ID of the material and the details to be modified.

Searching Within the Repository: The class enables a search function where users can enter an attribute and a corresponding value to locate specific materials.

Sorting Repository Materials: The class provides a method to sort materials based on a chosen attribute, enhancing the usability and accessibility of the repository. (users can choose whether the order goes descending or ascending)

Batch Addition of Materials: The class supports the simultaneous addition of multiple materials, thereby facilitating extensive data entry with ease.

Marking Materials as Favorite: The class includes a function to mark a material as a favorite, enhancing quick access for users.

Generating Repository Reports: The class enables the creation of comprehensive reports about the repository, providing valuable insights into the collection.

Managing User Interactions: The class features a run method, which serves as the main interaction point for users. It continuously prompts the user to choose an operation and acts accordingly.

# Summary of the Data Input and Error Handling in Scientific Repository User Interface

Our Scientific Repository Interface incorporates detailed data input, validation, and error handling mechanisms. The example of adding a new scientific material, where the system validates each attribute and handles duplication errors, is just a part of this comprehensive system. 

The interface is designed to handle a wide range of scenarios and edge cases, ensuring that it responds correctly and reasonably to all user inputs. It provides meaningful error messages and guides users to correct their inputs, ensuring an error-free experience. 

This robust design makes our Scientific Repository Interface a reliable tool for managing scientific materials effectively and efficiently.

In [ ]:
from tabulate import tabulate

class ScientificRepositoryUI:
    def __init__(self, repo_file):
        self.repo = ScientificRepository(repo_file)  # Initialize the repository
        self.next_id = self.calculate_next_id()  # Calculate the next ID

    def calculate_next_id(self):
        repository = self.repo.get_all_repository()  # Get all materials from the repository
        if repository:
            max_id = max(int(material['id']) for material in repository)  # Find the maximum ID
            return max_id + 1  # Increment the maximum ID
        else:
            return 1  # If the repository is empty, start from ID 1

    def run(self):
        while True:
            print("\nScientific Repository")
            print("--------------------")
            # Display the menu options
            print("1. Display all materials")
            print("2. Add a material")
            print("3. Remove a material")
            print("4. Update a material")
            print("5. Search for a material")
            print("6. Sort materials")
            print("7. Batch add materials")
            print("8. Mark a material as favorite")
            print("9. Generate report")
            print("10. Exit")
            choice = input("Please enter your choice: ")

            if choice.isdigit():  # Check if the input is a number
                choice = int(choice)  # Convert the input to an integer
                try:
                    # Depending on the choice, call the appropriate method
                    if choice == 1:
                        self.display_materials()
                    elif choice == 2:
                        self.add_material()
                    elif choice == 3:
                        self.remove_material()
                    elif choice == 4:
                        self.update_material()
                    elif choice == 5:
                        self.search_material()
                    elif choice == 6:
                        self.sort_materials()
                    elif choice == 7:
                        self.batch_add_materials()
                    elif choice == 8:
                        self.mark_as_favorite()
                    elif choice == 9:
                        self.print_report()
                    elif choice == 10:
                        # Confirm before exiting
                        if input('Are you sure you want to exit? (yes/no): ').lower() == 'yes':
                            break
                    else:
                        print("Invalid choice. Please enter a number between 1 and 10.")
                except Exception as e:
                    print(f"An error occurred: {e}")  # Print any errors that occur
            else:
                print("Invalid input. Please enter a number.")  # If the input is not a number, print an error message
        

    def display_materials(self):
        try:
            materials = self.repo.get_all_repository()

            if materials:
                # Convert the list of dictionaries to a list of lists for tabulate
                table_data = [list(material.values()) for material in materials]
                # Get the headers (keys) from the first dictionary in the materials list
                headers = list(materials[0].keys())

                print("\nMaterials:")
                print(tabulate(table_data, headers=headers, tablefmt='pipe'))  # Use 'pipe' format for Markdown-like tables
            else:
                print('No materials to display.')
        except Exception as e:
            print(f"An error occurred: {str(e)}")

    def add_material(self):
        try:
            id = self.next_id  # Use the next available ID
            self.next_id += 1  # Increment the next available ID

            print("Material Type:\n1. Book\n2. Article\n3. Journal\n4. Thesis\n5. Dissertation\n6. Conference Paper\n7. Report\n8. Patent\n9. Dataset\n10. Software")
            type_option = input("Enter the material type (choose a number from 1-10 or 'q' to quit): ")
            if type_option.lower() == 'q':
                return
            elif type_option.isdigit() and 1 <= int(type_option) <= 10:
                type = ['Book', 'Article', 'Journal', 'Thesis', 'Dissertation', 'Conference Paper', 'Report', 'Patent', 'Dataset', 'Software'][int(type_option)-1]
            else:
                print("Invalid input. Please enter a number between 1 and 10.")
                return

            title = input("Enter the material title (or 'q' to quit): ")
            if title.lower() == 'q':
                return
            elif title.strip() == '':
                print("Invalid input. Title cannot be empty.")
                return

            authors = input("Enter the author(s), separated by commas (or 'q' to quit): ")
            if authors.lower() == 'q':
                return

            publication_date = input("Enter the publication date (e.g., '2023-11-12' or 'q' to quit): ")
            if publication_date.lower() == 'q':
                return

            keywords = input("Enter keywords, separated by commas (or 'q' to quit): ")
            if keywords.lower() == 'q':
                return

            associated_field = input("Enter the associated field or discipline (or 'q' to quit): ")
            if associated_field.lower() == 'q':
                return

            material = {
                'id': id,
                'type': type,
                'title': title,
                'authors': [author.strip() for author in authors.split(',')],
                'publication_date': publication_date,
                'keywords': [keyword.strip() for keyword in keywords.split(',')],
                'associated_field': associated_field
            }

            if self.repo.material_exists(material):
                print("Material already exists in the repository.")
            else:
                self.repo.add_material(material)
                print(f"Material added successfully with ID: {id}!")
        except Exception as e:
            print(f"An error occurred: {str(e)}")
    
    def remove_material(self):
        id = input("Enter the ID of the material to remove: ")
        if id.isdigit():
            id = int(id)
            if self.repo.material_exists_by_id(id):  
                try:
                    self.repo.remove_material(id)
                    print("Material removed successfully!")
                except Exception as e:
                    print(f"An error occurred: {str(e)}")
            else:
                print("Invalid ID. The material does not exist.")
        else:
            print("Invalid ID. Please input a number.")
    

    def update_material(self):
        """
    Interacts with the user to update an existing material in the repository by ID and attribute.
    Handles attribute-specific input, formats and validation.
    """
        id = input("Enter the ID of the material you want to update (or 'q' to quit): ")
        if id.lower() == 'q':
            return
        if id.isdigit():
            id = int(id)
            if self.repo.material_exists_by_id(id):
                attribute_options = ["type", "title", "authors", "publication_date", "keywords", "associated_field", "remove favorite mark"]
                print("Select the attribute you want to update: ")
                for i, option in enumerate(attribute_options, start=1):
                    print(f"{i}. {option}")

                attribute_index = input("Enter the number of the attribute (or 'q' to quit): ")
                if attribute_index.lower() == 'q':
                    return
                if attribute_index.isdigit() and 1 <= int(attribute_index) <= len(attribute_options):
                    attribute = attribute_options[int(attribute_index) - 1]
                else:
                    print("Invalid selection. Please input a number from the list.")
                    return

                if attribute == "remove favorite mark":
                    new_value = False
                    attribute = "favorite"
                else:
                    new_value = input(f"Enter the new value for '{attribute}' (or 'q' to quit): ")
                    if new_value.lower() == 'q':
                        return
                    if attribute == "authors" or attribute == "keywords":
                        new_value = new_value.split(',')
                    elif attribute == "publication_date":
                        try:
                            datetime.strptime(new_value, '%Y-%m-%d')
                        except ValueError:
                            print("Invalid date. Please use format 'YYYY-MM-DD'")
                            return

                try:
                    success = self.repo.update_material(id, attribute, new_value)
                    if success:
                        print("Material updated successfully!")
                    else:
                        print("Update failed. Please make sure the ID is correct and try again.")
                except Exception as e:
                    print(f"An error occurred: {str(e)}")
            else:
                print("Invalid ID. The material does not exist.")
        else:
            print("Invalid ID. Please input a number.")


    def search_material(self):
        print("Enter the attribute to search by:")
        print("1. id")
        print("2. type")
        print("3. title")
        print("4. authors")
        print("5. publication_date")
        print("6. keywords")
        print("7. associated_field")
        print("8. favorite")

        attribute_options = {
            "1": "id",
            "2": "type",
            "3": "title",
            "4": "authors",
            "5": "publication_date",
            "6": "keywords",
            "7": "associated_field",
            "8": "favorite"
        }

        option = input("Please enter your choice: ")

        if option in attribute_options:
            key = attribute_options[option]

            try:
                if key == "id":  # Convert value to int if searching by id
                    value = int(input("Enter the value to search for: "))
                    results = self.repo.search_material(key, value)
                elif key == "favorite":  # Directly use search_favorites() function
                    results = self.repo.search_favorites()
                else:
                    value = input("Enter the value to search for: ")
                    results = self.repo.search_material(key, value)

                if results:
                    print("\nSearch Results:")
                    for material in results:
                        print(material)  # print the dictionary directly
                else:
                    print("No materials found matching the criteria.")
            except Exception as e:
                print(f"An error occurred: {e}")
        else:
            print("Invalid input. Please enter a number between 1 and 8.")

    def sort_materials(self):
        print("Enter the attribute to sort by:")
        print("1. id")
        print("2. type")
        print("3. title")
        print("4. authors")
        print("5. publication_date")
        print("6. keywords")
        print("7. associated_field")

        attribute_options = {
            "1": "id",
            "2": "type",
            "3": "title",
            "4": "authors",
            "5": "publication_date",
            "6": "keywords",
            "7": "associated_field",
        }

        option = input("Please enter your choice: ")

        if option in attribute_options:
            key = attribute_options[option]

            # Ask user for the sorting order
            print("Enter the sorting order:")
            print("1. Ascending")
            print("2. Descending")
            order_option = input("Please enter your choice: ")
            order = 'asc' if order_option == '1' else 'desc'

            try:
                sorted_materials = self.repo.sort_materials(key, order)  # Pass the order to sort_materials

                if sorted_materials:
                    # Convert the list of dictionaries to a list of lists for tabulate
                    table_data = [list(material.values()) for material in sorted_materials]
                    # Get the headers (keys) from the first dictionary in the materials list
                    headers = list(sorted_materials[0].keys())

                    print("\nSorted Materials:")
                    print(tabulate(table_data, headers=headers, tablefmt='pipe'))  # Use 'pipe' format for Markdown-like tables
                else:
                    print('No materials to display.')
            except Exception as e:
                print(f"An error occurred: {str(e)}")
        else:
            print("Invalid input. Please enter a number between 1 and 7.")

            
    def batch_add_materials(self):
        while True:
            num_materials = input("Enter the number of materials you want to add (or 'q' to quit): ")
            if num_materials.lower() == 'q':
                return
            elif num_materials.isdigit() and int(num_materials) > 0:
                num_materials = int(num_materials)
                break
            else:
                print("Please enter a positive integer.")

        materials = []
        for i in range(num_materials):
            print(f"\nAdding material {i+1} of {num_materials}")

            id = self.next_id  # Use the next available ID
            self.next_id += 1  # Increment the next available ID

            while True:
                print("Material Type:\n1. Book\n2. Article\n3. Journal\n4. Thesis\n5. Dissertation\n6. Conference Paper\n7. Report\n8. Patent\n9. Dataset\n10. Software")
                type_option = input("Enter the material type (choose a number from 1-10 or 'q' to quit): ")
                if type_option.lower() == 'q':
                    return
                elif type_option.isdigit() and 1 <= int(type_option) <= 10:
                    type = ['Book', 'Article', 'Journal', 'Thesis', 'Dissertation', 'Conference Paper', 'Report', 'Patent', 'Dataset', 'Software'][int(type_option)-1]
                    break
                else:
                    print("Invalid input. Please choose a number from 1-10.")

            title = input("Enter the material title (or 'q' to quit): ")
            if title.lower() == 'q':
                return
            elif title.strip() == '':
                print("Invalid input. Title cannot be empty.")
                continue

            authors = input("Enter the author(s), separated by commas (or 'q' to quit): ")
            if authors.lower() == 'q':
                return

            publication_date = input("Enter the publication date (e.g., '2023-11-12' or 'q' to quit): ")
            if publication_date.lower() == 'q':
                return

            keywords = input("Enter keywords, separated by commas (or 'q' to quit): ")
            if keywords.lower() == 'q':
                return

            associated_field = input("Enter the associated field or discipline (or 'q' to quit): ")
            if associated_field.lower() == 'q':
                return

            material = {
                'id': id,
                'type': type,
                'title': title,
                'authors': [author.strip() for author in authors.split(',')],
                'publication_date': publication_date,
                'keywords': [keyword.strip() for keyword in keywords.split(',')],
                'associated_field': associated_field
            }

            # Check for duplicate materials
            if not self.repo.material_exists(material):
                materials.append(material)
                print(f"Material {i+1} added to batch with ID: {id}!")
            else:
                print("This material already exists in the repository. Skipping...")

        try:
            self.repo.batch_add_materials(materials)
            print("Batch add successful!")
        except Exception as e:
            print(f"An error occurred: {str(e)}")


    def mark_as_favorite(self):
        try:
            id = input("Enter the ID of the material you want to mark as favorite: ")
            if id.isdigit():
                id = int(id)
                if self.repo.material_exists_by_id(id):
                    self.repo.mark_as_favorite(id)
                    print("Material marked as favorite!")
                else:
                    print("Material with the provided ID does not exist.")
            else:
                print("Invalid ID. Please input a number.")
        except Exception as e:
            print(f"An error occurred: {e}")
            
    def print_report(self):
        report = self.repo.generate_report()
        print(report)

# Run the user interface
ui = ScientificRepositoryUI("repository.json")
ui.run()

# Testing Overview

The below code includes a suite of unit tests for the ScientificRepository class. These tests are designed to ensure that each method in the class works as expected, and that the class can be used to manage a scientific repository effectively.

Each test creates a new instance of the ScientificRepository class and performs a set of operations. The results of these operations are then compared to expected outcomes to validate the implementation of each method.

The tests demonstrate a high level of coverage for the ScientificRepository class. However, it's important to note that due to the complexity of testing user interface input in the ScientificRepositoryUI class, manual testing has been performed for this class. This manual testing process has confirmed that the user interface is robust and error-free as of the last test cycle.

In [ ]:
# Testing Section
import uuid
import os
import unittest

class TestScientificRepository(unittest.TestCase):
    def setUp(self):
        self.filename = f'test_file_{uuid.uuid4().hex}.json'
        self.repo = ScientificRepository(self.filename)

    def tearDown(self):
        if os.path.exists(self.filename):
            os.remove(self.filename)

    def test_init(self):
        self.assertEqual(self.repo.filename, self.filename)
        self.assertEqual(self.repo.repository, [])

    def test_add_material(self):
        material = {
            'id': str(uuid.uuid4()),
            'title': 'Test Material',
            'type': 'book',
            'author': 'Test Author',
            'tags': ['test', 'material'],
            'favorite': False
        }
        self.repo.add_material(material)
        self.assertEqual(self.repo.repository[0], material)

    def test_search_material(self):
        material = {
            'id': str(uuid.uuid4()),
            'title': 'Test Material',
            'type': 'book',
            'author': 'Test Author',
            'tags': ['test', 'material'],
            'favorite': False
        }
        self.repo.add_material(material)
        result = self.repo.search_material('title', 'Test Material')
        self.assertEqual(result[0], material)


    def test_batch_add_materials(self):
        materials = [
            {
                'id': str(uuid.uuid4()),
                'title': 'Test Material 1',
                'type': 'book',
                'author': 'Test Author 1',
                'tags': ['test', 'material'],
                'favorite': False
            },
            {
                'id': str(uuid.uuid4()),
                'title': 'Test Material 2',
                'type': 'book',
                'author': 'Test Author 2',
                'tags': ['test', 'material'],
                'favorite': False
            }
        ]
        self.repo.batch_add_materials(materials)
        self.assertEqual(self.repo.repository, materials)

    def test_mark_as_favorite(self):
        material = {
            'id': str(uuid.uuid4()),
            'title': 'Test Material',
            'type': 'book',
            'author': 'Test Author',
            'tags': ['test', 'material'],
            'favorite': False
        }
        self.repo.add_material(material)
        self.repo.mark_as_favorite(material['id'])
        self.assertTrue(self.repo.repository[0]['favorite'])

    def test_search_favorites(self):
        material1 = {
            'id': str(uuid.uuid4()),
            'title': 'Test Material 1',
            'type': 'book',
            'author': 'Test Author 1',
            'tags': ['test', 'material'],
            'favorite': False
        }
        material2 = {
            'id': str(uuid.uuid4()),
            'title': 'Test Material 2',
            'type': 'book',
            'author': 'Test Author 2',
            'tags': ['test', 'material'],
            'favorite': True
        }
        self.repo.add_material(material1)
        self.repo.add_material(material2)
        favorites = self.repo.search_favorites()
        self.assertEqual(len(favorites), 1)
        self.assertEqual(favorites[0], material2)

    def test_sort_materials(self):
        material1 = {
            'id': str(uuid.uuid4()),
            'title': 'Test Material A',
            'type': 'book',
            'author': 'Test Author 1',
            'tags': ['test', 'material'],
            'favorite': False
        }
        material2 = {
            'id': str(uuid.uuid4()),
            'title': 'Test Material B',
            'type': 'book',
            'author': 'Test Author 2',
            'tags': ['test', 'material'],
            'favorite': True
        }
        self.repo.add_material(material1)
        self.repo.add_material(material2)
        sorted_materials = self.repo.sort_materials('title')
        self.assertEqual(sorted_materials, [material1, material2])

    def test_remove_material(self):
        material = {
            'id': str(uuid.uuid4()),
            'title': 'Test Material',
            'type': 'book',
            'author': 'Test Author',
            'tags': ['test', 'material'],
            'favorite': False
        }
        self.repo.add_material(material)
        self.repo.remove_material(material['id'])
        self.assertEqual(self.repo.repository, [])

    def test_update_material(self):
        material = {
            'id': str(uuid.uuid4()),
            'title': 'Test Material',
            'type': 'book',
            'author': 'Test Author',
            'tags': ['test', 'material'],
            'favorite': False
        }
        self.repo.add_material(material)
        self.repo.update_material(material['id'], 'title', 'Updated Material')
        self.assertEqual(self.repo.repository[0]['title'], 'Updated Material')

    def test_material_exists(self):
        material = {
            'id': str(uuid.uuid4()),
            'title': 'Test Material',
            'type': 'book',
            'author': 'Test Author',
            'tags': ['test', 'material'],
            'favorite': False
        }
        self.repo.add_material(material)
        exists = self.repo.material_exists(material)
        self.assertTrue(exists)

    def test_material_exists_by_id(self):
        material = {
            'id': str(uuid.uuid4()),
            'title': 'Test Material',
            'type': 'book',
            'author': 'Test Author',
            'tags': ['test', 'material'],
            'favorite': False
        }
        self.repo.add_material(material)
        exists = self.repo.material_exists_by_id(material['id'])
        self.assertTrue(exists)

run_tests(TestScientificRepository)